<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Naive_Bayes_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 4.2 MB 86.9 MB/s 
     |████████████████████████████████| 1.2 MB 75.2 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 81.1 MB/s 
     |████████████████████████████████| 6.6 MB 71.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=fd43d1ceca5fce3e635e60d83f8d396be923e0f65143699b097eea1470800f6b
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [4]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [5]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Riots,Feb Residents of Lezhe blocked the road to P...


In [6]:
#del data_set

In [7]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [8]:
train = model.encode(df.cleaned, device='cuda')

In [9]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [10]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[0.15234016, 0.116330884, -0.25540513, 0.13642...",3
1,"[0.03704272, 0.5278288, 0.11853743, 0.06128985...",3
2,"[-0.05260811, 0.22022387, -0.10819439, 0.40675...",0


In [11]:
 df_features.to_csv("./features_set.csv")

In [12]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5968223443041388
16.304254183999888


In [13]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

**Model Training**

Naive Bayes

In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [15]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB

gnb = GaussianNB()
gnb.fit(x_train,y_train)
print("Score of GaussianNB",gnb.score(x_test,y_test))

Score of GaussianNB 0.82695


In [16]:
y_pred = gnb.predict(x_test)

In [17]:
print(y_pred)
print(y_test)

[0 3 0 ... 5 2 2]
75721    0
80184    3
19864    0
76699    3
92991    1
        ..
32595    3
29313    2
37862    5
53421    2
42410    2
Name: event_type, Length: 20000, dtype: int64


In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      4839
           1       0.85      0.68      0.76      2591
           2       0.96      0.96      0.96      6997
           3       0.68      0.72      0.70      1631
           4       0.65      0.72      0.68      1098
           5       0.71      0.79      0.74      2844

    accuracy                           0.83     20000
   macro avg       0.77      0.78      0.77     20000
weighted avg       0.83      0.83      0.83     20000



K FOLD

In [19]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[3882  202    0  103  102  550]
 [ 617 1768    0   15   90  101]
 [  15    0 6688  206   61   27]
 [  30    2  220 1167   51  161]
 [  53   50   40   68  793   94]
 [ 246   53   13  168  123 2241]]


In [20]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = gnb, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

0.004264607250380756